(2) We're going to simulate the attitude dynamics of a rigid body like this T-wrench: https://www.youtube.com/watch?v=1n-HMSCDYtM

In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [96]:
using LinearAlgebra
using ForwardDiff
using DifferentialEquations
using Plots
plotly()

In [248]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I + hat(q[2:4])]
end

function R(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I - hat(q[2:4])]
end

H = [zeros(1,3); I]

4×3 Matrix{Float64}:
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [ ]:
#Inertia matrix we'll be using
J = Diagonal([1.0; 2.0; 3.0])

In [172]:
#Implement Euler's equation for a rigid body
function dynamics_ω(ω)
    ω̇ = -J\(hat(ω)*J*ω)
end

dynamics_ω (generic function with 1 method)

In [173]:
#Implement the full attitude dynamics using a rotation matrix
function dynamics_Q(x)
    Q = reshape(x[1:9],3,3)
    ω = x[10:12]
    
    Q̇ = Q*hat(ω)
    
    ẋ = [Q̇[:]; dynamics_ω(ω)]
end

function dynamics_Q!(ẋ,x,p,t)
    ẋ .= dynamics_Q(x)
end

dynamics_Q! (generic function with 1 method)

In [174]:
#Implement the full attitude dynamics using a quaternion

function dynamics_q(x)
    q = x[1:4]
    ω = x[5:7]
    
    q̇ = 0.5*L(q)*H*ω
    
    ẋ = [q̇; dynamics_ω(ω)]
end

function dynamics_q!(ẋ,x,p,t)
    ẋ .= dynamics_q(x)
end

dynamics_q! (generic function with 1 method)

In [249]:
#Initial Conditions
Q0 = Array(I(3))
q0 = [1.0; 0; 0; 0]
ω0 = [0; 2*pi; 0] + 1e-2*randn(3) #small perturbation from intermediate-axis spin

x0Q = [Q0[:]; ω0]
x0q = [q0; ω0]

7-element Vector{Float64}:
  1.0
  0.0
  0.0
  0.0
 -0.006564671783207886
  6.288999068377863
 -0.004195341280787257

In [252]:
#Set up and solve with DifferentialEquations.jl with default settings
Tf = 60.0
tspan = (0.0, Tf)

prob_q = ODEProblem(dynamics_q!, x0q, tspan)
prob_Q = ODEProblem(dynamics_Q!, x0Q, tspan)

sol_q = solve(prob_q,Tsit5());
sol_Q = solve(prob_Q,Tsit5());

In [253]:
#ToDo: Make a meshcat animation of the T-wrench here
plot(sol_q, vars=(5,6,7))

In [254]:
#Plot the error in the quaternion norm vs. time
qnorm = zeros(length(sol_q.t))
for k = 1:length(sol_q.t)
    qnorm[k] = norm(sol_q.u[k][1:4])
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="",lw=2)

In [255]:
#Plot the orthogonality error vs. time for the rotation matrix
Qerr = zeros(length(sol_Q.t))
for k = 1:length(sol_Q.t)
    Qk = reshape(sol_Q.u[k][1:9],3,3)
    Qerr[k] = tr(Qk'*Qk - I(3))
end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="",lw=2)

If you're writing your own integrator, you can easily re-normalize the quaternion at each step. Another option that can work if you're using an off-the-shelf ODE solver is to add proportional feedback on the quaternion norm error to the dynamics. This is an example of an idea called Baumgarte stabilization that can be used to make Runge-Kutta methods better respect known constraints and conserved quantities. Implement it and test some feedback gains between 1 and 20 before choosing a value that you find works best.

In [345]:
function dynamics_q_stabilized(x)
    q = x[1:4]
    ω = x[5:7]
    
    Kp = 12.0
    
    q̇ = 0.5*L(q)*H*ω - Kp*(norm(q)-1).*q
    ω̇ = -J\(hat(ω)*J*ω)
    
    ẋ = [q̇; ω̇]
end

function dynamics_q_stabilized!(ẋ,x,p,t)
    ẋ .= dynamics_q_stabilized(x)
end

dynamics_q_stabilized! (generic function with 1 method)

In [346]:
prob_q2 = ODEProblem(dynamics_q_stabilized!, x0q, tspan)
sol_q2 = solve(prob_q2,Tsit5());

In [347]:
#Plot norm(q) with and without stabilization
qnorm2 = zeros(length(sol_q2.t))
for k = 1:length(sol_q2.t)
    qnorm2[k] = norm(sol_q2.u[k][1:4])
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="No Stabilization",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)

Unfortunately, the stabilization idea doesn't work as well with rotation matrices since the constraints are much more complicated. Instead, we can implement a Runge-Kutta method that automatically preserves orthogonality of rotation matrices (or normalization of quaternions) by using the exponential map and group multiplication operations. Let's do this for the rotation matrix case first by modifying the explicit midpoint method. This is a simple example of what's known as a Runge-Kutta-Munthe-Kaas method. It is possible to construct an RK-MK method that stays on a Lie group from any standard RK method.

In [236]:
function lie_midpoint_step_Q(xk,h)
    
    Qk = reshape(xk[1:9],3,3)
    ωk = xk[10:12]
    
    ω̇k = dynamics_ω(ωk)
    ωm = ωk+0.5*h*ω̇k
    ω̇m = dynamics_ω(ωm)
    
    ωn = ωk + h*ω̇m
    
    ΔQ = exp(hat(h*ωm))
    
    Qn = Qk*ΔQ
    
    return [Qn[:]; ωn]
end

lie_midpoint_step_Q (generic function with 1 method)

In [353]:
#Now let's simulate the same example and plot the orthogonality error again

h = 0.1
N = N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1))

xtrajQ = zeros(12,N)
xtrajQ[:,1] .= x0Q

for k = 1:(N-1)
    xtrajQ[:,k+1] .= lie_midpoint_step_Q(xtrajQ[:,k], h)
end

Qerr2 = zeros(N)
for k = 1:N
    Qk = reshape(xtrajQ[1:9,k],3,3)
    Qerr2[k] = tr(Qk'*Qk - I(3))
end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="RK",lw=2)
plot!(thist,Qerr2,label="Lie Group",lw=2)

Now let's implement the same Lie group idea but use quaternions instead of rotation matrices. To do this, we'll also need to implement the quaternion exponential map. See if you can figure out a way to implement it without introducing divide-by-zero issues when the angular velocity goes to zero (hint: look into the sinc function).

In [354]:
function Expq(ϕ)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
end

function lie_midpoint_step_q(xk,h)
    
    qk = xk[1:4]
    ωk = xk[5:7]
    
    ω̇k = dynamics_ω(ωk)
    ωm = ωk+0.5*h*ω̇k
    ω̇m = dynamics_ω(ωm)
    
    ωn = ωk + h*ω̇m
    
    Δq = Expq(h*ωm)
    
    qn = L(qk)*Δq
    
    return [qn; ωn]
end

lie_midpoint_step_q (generic function with 1 method)

In [358]:
#Now let's simulate and plot the quaternion norm again

xtrajq = zeros(7,N)
xtrajq[:,1] .= x0q

for k = 1:(N-1)
    xtrajq[:,k+1] .= lie_midpoint_step_q(xtrajq[:,k], h)
end

qnorm3 = zeros(N)
for k = 1:N
    qnorm3[k] = norm(xtrajq[1:4,k])
end

plot(sol_q.t,qnorm,label="No Stabilization",xlabel="Time (sec)",ylabel="||q||",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)
plot!(thist,qnorm3,label="Lie Group",lw=2)